In [1]:
## AE Adaptive Thresholding Experiment Notebook
# 1. Connect to Drive
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/projects/ae-vae-anomaly-detection/data/processed/cleaned.csv'

Mounted at /content/drive


In [2]:
# 2. Load Processed Data
import pandas as pd

#load cleaned data
cleaned_df = pd.read_csv(data_path)
print(f'Loaded cleaned data: {cleaned_df.shape[0]} rows, {cleaned_df.shape[1]} columns')

# get a small sample for training model
df = cleaned_df.sample(frac=1, random_state=42)
print(f'Sampled data: {df.shape[0]} rows, {df.shape[1]} columns')

Loaded cleaned data: 640788 rows, 178 columns
Sampled data: 640788 rows, 178 columns


In [3]:
# 3. Train/Test Split
from sklearn.model_selection import train_test_split

# Separate features and label
X = df.drop(columns='label')
y = df['label']

# First split off a mixed test set (normal+anomaly), stratified to keep the same
#    anomaly ratio in train and test.
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# From the remaining (X_temp / y_temp) we only train on the normal samples:
#    i.e. drop the anomalies from this “train+val pool”
X_train_val = X_temp[y_temp == 0]

# Finally split that normal‐only pool into X_train / X_val
X_train, X_val = train_test_split(
    X_train_val,
    test_size=0.2,
    random_state=42
)

# Now:
#  - X_train, X_val are both *only* normal samples (for AE/VAE training & early stopping)
#  - X_test / y_test is your held‐out mixed set (with both normal and anomalous),
#    and it never overlaps with X_train or X_val

# 从 X_test 中划出验证集（20%），用于寻找最优阈值
X_eval, X_test_remain, y_eval, y_test_remain = train_test_split(
    X_test, y_test, test_size=0.8, stratify=y_test, random_state=42
)

In [5]:
!mkdir -p /content/src/models

In [6]:
import sys
sys.path.append('/content/src')
from models.ae_model import build_ae, train_autoencoder_with_optimizer, plot_training_history
from models.ae_evaluation import evaluate_anomaly_detection, visualize_reconstruction, compute_reconstruction_error
from models.thresholding import find_best_f1_threshold, apply_threshold

In [7]:
from keras.optimizers import AdamW
ae = build_ae(input_dim=X_train.shape[1], encoding_dim=16, activation='tanh')
history, model = train_autoencoder_with_optimizer(
    ae,
    X_train,
    X_val,
    optimizer=AdamW(learning_rate=1e-3),
    save_path='best_ae.h5'
)

Epoch 1/100
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0272 - mae: 0.0667

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step - loss: 0.0272 - mae: 0.0667 - val_loss: 7.0537e-04 - val_mae: 0.0062 - learning_rate: 0.0010
Epoch 2/100
6259/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0013 - mae: 0.0097

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 32s 4ms/step - loss: 0.0013 - mae: 0.0097 - val_loss: 4.8359e-04 - val_mae: 0.0051 - learning_rate: 0.0010
Epoch 3/100
6260/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 - mae: 0.0088

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 0.0010 - mae: 0.0088 - val_loss: 4.1739e-04 - val_mae: 0.0046 - learning_rate: 0.0010
Epoch 4/100
6257/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 - mae: 0.0085

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - loss: 0.0010 - mae: 0.0085 - val_loss: 4.1037e-04 - val_mae: 0.0046 - learning_rate: 0.0010
Epoch 5/100
6263/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.7695e-04 - mae: 0.0083

...

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 9.0571e-04 - mae: 0.0079 - val_loss: 3.1171e-04 - val_mae: 0.0034 - learning_rate: 3.1250e-05
Epoch 20/100
6258/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.0505e-04 - mae: 0.0079

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 9.0505e-04 - mae: 0.0079 - val_loss: 3.1143e-04 - val_mae: 0.0034 - learning_rate: 3.1250e-05
Epoch 21/100
6258/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.0331e-04 - mae: 0.0079
Epoch 21: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 9.0331e-04 - mae: 0.0079 - val_loss: 3.1331e-04 - val_mae: 0.0035 - learning_rate: 3.1250e-05
Epoch 22/100
6257/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.0978e-04 - mae: 0.0080

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 9.0977e-04 - mae: 0.0080 - val_loss: 3.1047e-04 - val_mae: 0.0034 - learning_rate: 1.5625e-05
Epoch 23/100
6257/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.0237e-04 - mae: 0.0079

6266/6266 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - loss: 9.0237e-04 - mae: 0.0079 - val_loss: 3.0897e-04 - val_mae: 0.0034 - learning_rate: 1.5625e-05
Epoch 24/100
6265/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.0516e-04 - mae: 0.0080


Epoch 24: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - loss: 9.0516e-04 - mae: 0.0080 - val_loss: 3.0626e-04 - val_mae: 0.0034 - learning_rate: 1.5625e-05
Epoch 25/100
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step - loss: 8.9713e-04 - mae: 0.0079 - val_loss: 3.0767e-04 - val_mae: 0.0034 - learning_rate: 7.8125e-06
Epoch 26/100
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - loss: 9.0088e-04 - mae: 0.0079 - val_loss: 3.0728e-04 - val_mae: 0.0034 - learning_rate: 7.8125e-06
Epoch 27/100
6258/6266 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.0058e-04 - mae: 0.0079
Epoch 27: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - loss: 9.0058e-04 - mae: 0.0079 - val_loss: 3.0996e-04 - val_mae: 0.0034 - learning_rate: 7.8125e-06
Epoch 28/100
6266/6266 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - loss: 9.0470e-04 - mae: 0.0079 - val_loss: 3.0927e-04 - val_mae: 0.0034 - learning_rate

In [13]:
from src.models.thresholding import find_best_f1_threshold
from importlib import reload
import src.models.thresholding as thresholding
reload(thresholding)

<module 'src.models.thresholding' from '/content/src/models/thresholding.py'>

In [9]:
eval_errors = compute_reconstruction_error(model, X_eval)
best_threshold, f1_scores = find_best_f1_threshold(eval_errors, y_eval)
print(f"\n📍 Best Threshold from Val Set: {best_threshold:.6f}")

801/801 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

📍 Best Threshold from Val Set: 0.004045


In [10]:
from sklearn.metrics import classification_report, confusion_matrix
test_errors = compute_reconstruction_error(model, X_test_remain)
y_pred = apply_threshold(test_errors, best_threshold)
print("\n📊 Classification Report (Test Set):")
print(classification_report(y_test_remain, y_pred, digits=4))
print("\n🧮 Confusion Matrix:")
print(confusion_matrix(y_test_remain, y_pred))

3204/3204 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step

📊 Classification Report (Test Set):
              precision    recall  f1-score   support

         0.0     0.9996    0.9852    0.9924    100242
         1.0     0.6034    0.9847    0.7483      2285

    accuracy                         0.9852    102527
   macro avg     0.8015    0.9850    0.8703    102527
weighted avg     0.9908    0.9852    0.9870    102527


🧮 Confusion Matrix:
[[98763  1479]
 [   35  2250]]


In [8]:
basic_error = compute_reconstruction_error(model, X_test)
evaluate_anomaly_detection(basic_error, y_test)

4005/4005 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step


{'threshold': np.float64(0.004072101215107635),
 'precision': np.float64(0.6089590997619563),
 'recall': np.float64(0.9852941176470589),
 'f1': np.float64(0.7527083054228946),
 'roc_auc': np.float64(0.9907858272676521),
 'confusion_matrix': {'TP': np.int64(2814),
  'FP': np.int64(1807),
  'TN': np.int64(123495),
  'FN': np.int64(42)}}

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
